# Problem Description.

This project is based in the sections about clingo applications
and multi-shot solving of the paper [1].
Before starting, we recommend you to read them carefully.
The idea of the project comes from the paper [2].

* [1] Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2020). How to build your own ASP-based system?! CoRR, abs/2008.06692.
* [2] El-Kholany, M. M. S. & Gebser, M. (2020). Job Shop Scheduling with Multi-shot ASP. TAASP 2020.
    Available at: http://www.kr.tuwien.ac.at/events/taasp20/papers/TAASP_2020_paper_4.pdf

The task of this project is to solve the Job Shop problem in ASP using multi-shot solving.
For this, you have to write a clingo application in `Python` and an ASP encoding.

This project extends the project on `scheduling` that you can find at the parent directory.
Please finish that project before starting with this one.
In that project you had to write an ASP encoding for solving the Job Shop problem.
In this project you will extend your previous work to improve its performance.

In the Job Shop problem every job consists of some operations that have to be performed in a given sequence.
In this project, we assign to every operation its earliest starting point,
i.e., the sum of processing times of predecessor operations belonging to the same job.
The operations of a job could be processed at their earliest starting times if the machines were free.
However, machines may be occupied by operations of other jobs,
so that the earliest starting times merely provide a lower bound on the actual starting times of the operations.

Given some facts about the predicates `duration/3` and `sequence/3`
describing an instance (see the `scheduling` project for more information)
the earliest starting point `E` of the operation of job `J` on machine `M`
can be defined by the following rule:
```
est(J,M,E) :- sequence(J,M,S), E = #sum{DD,MM,SS : sequence(J,MM,SS), SS<S, duration(J,MM,DD)}.
```
 
In this project, we will split the set of all operations into time windows based on the earliest starting time.
Given an instance with `o` operations and `w` time windows,
the `i`-th time window for `1 <= i < w`
includes the operations at positions from `(i-1)*o/w + 1` to `i*o/w` (where `/` represents the integer division)
and the `w`-th time window includes the operations at positions from `(i-1)*o/w + 1` to `o`.

Using this decomposition, the operations have to be scheduled window wise, using multi-shot solving.
First, the operations for time window `1` must be optimally scheduled (solving a logic program).
Then, those operations should be fixed (using the functions about externals), and 
the operations for time window `2` must be optimally scheduled (again, solving a logic program).
The process should continue in this way until the operations of all time windows have been scheduled.

Note that the approach does not guarantee to find an schedule that is globally optimal.
This is the price that we pay for the expected improvement of the overall solving time.

In the following, we give some hints on the ASP encoding and the `clingo` application.

## Hints on the ASP encoding.

In your ASP encoding, we suggest to define a predicate `window/1`
representing the current time window `i`:
```
window(I)  % The current time window is I
```
Then the answer sets that do not have exactly one time window can be excluded with this constraint:
```
:- { window(I) } != 1.
```

You can select the operations of the current or previous time windows with this rule:
```
select(J,M) :- est(J,M,E),
               Pos = #count{JJ,MM: est(JJ,MM,EE), EE<E},
               O   = #count{JJ,MM: duration(JJ,MM,DD)},
               window(I),
               Pos <= (I*O)/w.
```
where `w` is the constant representing the number of time windows.

Then the answer sets that schedule some operation of a posterior time window
can be excluded with this constraint:
```
:- perm(J,M,P), not select(J,M).
```
where, as in the `scheduling` project, 
`perm(J,M,P)` is the output predicate representing that
job `J` on machine `M` occurs at position `P`.

The rest of the encoding should:
* generate the atoms of the predicate `window/1` (possibly using external declarations),
* generate the atoms of the predicate `perm/3` (possibly using external declarations),
* adapt your encoding of the `scheduling` project to this new setting (where some operations may be not scheduled).

## Hints on the `clingo` application.

The application only needs to ground the encoding once.

For every time window `i` where `1 <= i <= w`, the application
has to establish that `window(i)` must be true,
and compute an optimal model of the new encoding
(that prohibits the scheduling of operations of posterior time windows).

When `i>1`, the application also has to establish that
the atoms of predicate `perm/3` of the previous time window `i-1` must be true.

In the next cell you can find a template for the `clingo` application.
In its current form, it solves a program `w` times, 
where `w` is a constant of the logic program, 
and prints the last optimal model.

In [3]:
%%file multi-jobshop.py
import sys
import clingo

class MultiJobShopApp(clingo.Application):
    program_name = "multi-jobshop"
    version = "1.0"

    def __init__(self):
        self._model = None

    def _on_model(self, model):
        if model.optimality_proven:
            self._model = model.symbols(shown=True)

    def main(self, ctl, files):
        for path in files: ctl.load(path)
        if not files:
            ctl.load("-")
        ctl.ground([("base", [])], context=self)
        # for every time window
        for window in range(1, ctl.get_const("w").number+1):
            # do something else...
            ctl.solve(on_model=self._on_model)
        if self._model:
            print("Answer:\n{}".format(" ".join([str(atom) for atom in self._model])))

if __name__ == "__main__":
    clingo.clingo_main(MultiJobShopApp(), sys.argv[1:])

Overwriting multi-jobshop.py


Given the following logic program:

In [2]:
%%file example.lp
{ a(X) : X=1..w } = 1.
#minimize{ 1,X : a(X) }.

Overwriting example.lp


the next call computes an optimal model `3` times and prints the result of the last computation:

In [ ]:
! python multi-jobshop.py example.lp --outf=3 --opt-mode=optN 1 -c w=3

Option `--outf=3` eliminates the original `clingo` output. Options `--opt-mode=optN 1` tell `clingo` to compute one optimal model in each `solve` call. Options `-c w=3` establishes the value of constant `w` to `3`.

# Representation in ASP.

The representation of the instances and the solutions is the same as
that of the Job Shop problem from the `scheduling` project.

# Task.

Write a clingo application in `Python` and an ASP encoding for solving the Job Shop problem
using the multi-shot approach described above.
Given an instance and the ASP encoding, the application should
print one answer to the problem.

# Framework.

In the directory ``instances`` you can find 16 instances.
They are the same as the ones for the `scheduling` project.

You have to submit two files named ``multi-jobshop.py`` and ``multi-jobshop.lp``,
included as templates in this directory.
The second one must contain the line
```
#show perm/3.
```
and no more ``#show`` statements, so that in the output only the corresponding atoms appear.

The output should follow the format of the application template, 
i.e., using option `--outf=3` it should only print two lines:
the first with the string `Answer:` 
and the second with the atoms of the solution in the usual format.

We will test your submission using the following command line:
```
python multi-jobshop.py multi-jobshop.lp instance.lp --outf=3 --opt-mode=optN 1 -c w=3
```
where `instance.lp` is one instance. 
We will always set the value of `w` to `3`, but your code should work for any value of `w` greater than `0`.

Click the link at the following cell to download a zip file with the instances and the templates.·
We recommend you to work with them in your own computer, using your own installation of ``clingo``.

In [1]:
from IPython.display import FileLink
FileLink("multi-shot.zip")

/home/jovyan/advanced/multi-shot/multi-shot.zip

Alternatively, you can also run your application in the next cells, but this is not the recommended option. If you work in this notebook, remember to download the files that you modify to your computer, otherwise you will lose your changes.

In [4]:
%%file multi-jobshop.lp

% The earliest starting point of operation of job J on machine M is E
est(J,M,E) :- sequence(J,M,S), E = #sum{DD,MM,SS : sequence(J,MM,SS), SS<S, duration(J,MM,DD)}.

% Exclude answers that do not contain exactly one window
:- { window(I) } != 1.
    
% Select the operations of job J on machine M if it belongs to the current or previous time windows
select(J,M) :- est(J,M,E),
               Pos = #count{JJ,MM: est(JJ,MM,EE), EE<E},
               O   = #count{JJ,MM: duration(JJ,MM,DD)},
               window(I),
               Pos <= (I*O)/w.

% Exclude answers that schedule some operation that does not belong to the current or previous time windows            
:- perm(J,M,P), not select(J,M).            
            
        
% Your encoding please...            
        
    
#show perm/3.

Writing multi-jobshop.lp


# Formalities.
You can work on the solution alone or in groups of two people.
Different groups have to submit different solutions, in case
of plagiarism all groups involved will fail the project.

We will test your encoding with the provided instances.
Your solution has to correctly print one solution for every instance.
This will be tested automatically.
Please contact us if you get stuck.

# Tips:

* If `w` is `1` then the approach reduces to the original Job Shop problem of the previous project. You can check that in that case you obtain the same result with the current approach and with your solution to the other project.

* If you are stuck you can contact us. We will do out best to answer all your questions. You can send us questions and remarks either via Moodle or by email.

* Start as soon as possible to avoid running out of time. However, if you still realize that you have problems making it before the deadline, please contact us instead of copying another solution.
